In [ ]:
import pandas as pd
import numpy as np
import imageio
import cv2
import matplotlib.pyplot as plt
import os
from pathlib import Path
import re
from PIL import Image
import re
from collections import defaultdict

from colour_demosaicing import (
    ROOT_RESOURCES_EXAMPLES,
    demosaicing_CFA_Bayer_bilinear,
    demosaicing_CFA_Bayer_Malvar2004,
    demosaicing_CFA_Bayer_Menon2007,
    mosaicing_CFA_Bayer)

In [ ]:
from RawHandler.RawHandler import RawHandler
from RawHandler.utils import linear_to_srgb
from src.training.load_config import load_config

def apply_gamma(x, gamma=2.2):
    return x ** (1 / gamma)

def reverse_gamma(x, gamma=2.2):
    return x ** gamma

In [ ]:
run_config = load_config()
raw_path = Path(run_config['base_data_dir'])
outpath = Path(run_config['cropped_raw_subdir'])
alignment_csv =  outpath / run_config['align_csv']
colorspace = run_config['colorspace']
crop_size = run_config['cropped_raw_size']
file_list = os.listdir(raw_path)

In [ ]:
def pair_images_by_scene(file_list, min_iso=100):
    """
    Given a list of RAW image file paths:
      1. Extract ISO from filenames
      2. Remove files with ISO < min_iso
      3. Group by scene name
      4. Pair each image with the lowest-ISO version of the scene

    Args:
        file_list (list of str): Paths to RAW files
        min_iso (int): Minimum ISO to keep (default=100)

    Returns:
        dict: {scene_name: [(img_path, gt_path), ...]}
    """
    iso_pattern = re.compile(r"_ISO(\d+)_")
    scene_pairs = {}

    # Step 1: Extract iso and scene
    images = []
    for path in file_list:
        filename = os.path.basename(path)
        match = iso_pattern.search(filename)
        if not match:
            continue  # skip if no ISO
        iso = int(match.group(1))
        if iso < min_iso:
            continue  # filter out low ISOs

        # Extract scene name:
        if "_GT_" in filename:
            scene = filename.split("_GT_")[0]
        else:
            # Scene = part before "_ISO"
            scene = filename.split("_ISO")[0]
        if 'X-Trans' in filename:
            continue

        images.append((scene, iso, path))

    # Step 2: Group by scene
    grouped = defaultdict(list)
    for scene, iso, path in images:
        grouped[scene].append((iso, path))

    # Step 3: For each scene, pick lowest ISO as GT
    for scene, iso_paths in grouped.items():
        iso_paths.sort(key=lambda x: x[0])  # sort by ISO ascending
        gt_iso, gt_path = iso_paths[0]      # lowest ISO ≥ min_iso
        pairs = [(path, gt_path) for iso, path in iso_paths if path != gt_path]
        scene_pairs[scene] = pairs

    return scene_pairs


In [ ]:
pair_file_list = pair_images_by_scene(file_list)

In [ ]:
def get_file(impath, crop_size=crop_size):
        rh = RawHandler(impath)
        
        width, height = rh.raw.shape

        # Check if the image is large enough to be cropped.
        if width < crop_size or height < crop_size:
            im = rh.apply_colorspace_transform(colorspace=colorspace)
        else:
            # Calculate the coordinates for the center crop.
            left = (width - crop_size) // 2
            top = (height - crop_size) // 2

            # Ensure the top-left corner is on an even pixel coordinate for Bayer alignment.
            if left % 2 != 0:
                left -= 1
            if top % 2 != 0:
                top -= 1
            
            # Calculate the bottom-right corner based on the adjusted top-left corner.
            # Since crop_size is even, right and bottom will also be even.
            right = left + crop_size
            bottom = top + crop_size
        return rh.raw[left:right, top:bottom], rh

In [ ]:
from tqdm import tqdm

In [ ]:
from pidng.core import RAW2DNG, DNGTags, Tag
from pidng.defs import *

def get_ratios(string, rh):
    return [x.as_integer_ratio() for x in rh.full_metadata[string].values]


def rational_wb(rh, denominator=1000):
    wb = np.array(rh.core_metadata.camera_white_balance)
    numerator_matrix = np.round(wb * denominator).astype(int)
    return [[num, denominator] for num in numerator_matrix]
def convert_ccm_to_rational(matrix_3x3, denominator=10000):

    numerator_matrix = np.round(matrix_3x3 * denominator).astype(int)
    numerators_flat = numerator_matrix.flatten()
    ccm_rational = [[num, denominator] for num in numerators_flat]
    
    return ccm_rational


def get_as_shot_neutral(rh, denominator=10000):

    cam_mul = rh.core_metadata.camera_white_balance
    
    if cam_mul[0] == 0 or cam_mul[2] == 0:
        return [[denominator, denominator], [denominator, denominator], [denominator, denominator]]

    r_neutral = cam_mul[1] / cam_mul[0]
    g_neutral = 1.0 
    b_neutral = cam_mul[1] / cam_mul[2]

    return [
        [int(r_neutral * denominator), denominator],
        [int(g_neutral * denominator), denominator],
        [int(b_neutral * denominator), denominator],
    ]


def to_dng(uint_img, rh, filepath):
    width = uint_img.shape[1]
    height = uint_img.shape[0]
    bpp = 16

    exposures = get_ratios('EXIF ExposureTime', rh)
    fnumber = get_ratios('EXIF FNumber', rh)
    ExposureBiasValue = get_ratios('EXIF ExposureBiasValue', rh) 
    FocalLength = get_ratios('EXIF FocalLength', rh) 
    ccm1 = convert_ccm_to_rational(rh.core_metadata.rgb_xyz_matrix[:3, :])
    t = DNGTags()
    t.set(Tag.ImageWidth, width)
    t.set(Tag.ImageLength, height)
    t.set(Tag.TileWidth, width)
    t.set(Tag.TileLength, height)
    t.set(Tag.BitsPerSample, bpp)

    t.set(Tag.SamplesPerPixel, 1) 
    t.set(Tag.PlanarConfiguration, 1) 

    t.set(Tag.TileWidth, width)
    t.set(Tag.TileLength, height)
    t.set(Tag.Orientation, rh.full_metadata['Image Orientation'].values[0])
    t.set(Tag.PhotometricInterpretation, PhotometricInterpretation.Color_Filter_Array)
    t.set(Tag.CFARepeatPatternDim, [2,2])
    t.set(Tag.CFAPattern, CFAPattern.RGGB)
    bl = rh.core_metadata.black_level_per_channel
    t.set(Tag.BlackLevelRepeatDim, [2,2])
    t.set(Tag.BlackLevel, bl)
    t.set(Tag.WhiteLevel, rh.core_metadata.white_level)

    t.set(Tag.BitsPerSample, bpp)

    t.set(Tag.ColorMatrix1, ccm1)
    t.set(Tag.CalibrationIlluminant1, CalibrationIlluminant.D65)
    wb = get_as_shot_neutral(rh)
    t.set(Tag.AsShotNeutral, wb)
    t.set(Tag.BaselineExposure, [[0,100]])
    t.set(Tag.Make, rh.full_metadata['Image Make'].values)
    t.set(Tag.Model, rh.full_metadata['Image Model'].values)



    t.set(Tag.FocalLength, FocalLength)
    t.set(Tag.EXIFPhotoLensModel, rh.full_metadata['EXIF LensModel'].values)
    t.set(Tag.ExposureBiasValue, ExposureBiasValue)
    t.set(Tag.ExposureTime, exposures)
    t.set(Tag.FNumber, fnumber)
    t.set(Tag.PhotographicSensitivity, rh.full_metadata['EXIF ISOSpeedRatings'].values)
    t.set(Tag.DNGVersion, DNGVersion.V1_4)
    t.set(Tag.DNGBackwardVersion, DNGVersion.V1_2)
    t.set(Tag.PreviewColorSpace, PreviewColorSpace.Adobe_RGB)

    r = RAW2DNG()

    r.options(t, path="", compress=False)

    r.convert(uint_img, filename=filepath)

In [ ]:
for key in tqdm(pair_file_list.keys()):
    image_pairs = pair_file_list[key]
    for noisy, gt in image_pairs:
        try:
            bayer, rh = get_file(f'{raw_path}/{noisy}')
            noisy_path = outpath / (noisy)
            to_dng(bayer, rh, str(noisy_path))

            gt_path = outpath / (gt)
            if not os.path.exists(gt_path):
                bayer, rh = get_file(f'{raw_path}/{gt}')
                to_dng(bayer, rh, str(gt_path))
        except:
            print(f"Skipping {raw_path}/{noisy}, {raw_path}/{gt}")


In [ ]:
#Testing data is properly copied

In [ ]:
rhdng = RawHandler(str(outpath / "Bayer_MuseeL-sol-A7C-brighter_ISO100_sha1=18eaa9931d9a0f6f0511552ef6bf2fd040d82878.arw.dng"))
rh = RawHandler('/Volumes/EasyStore/RAWNIND/Bayer_MuseeL-sol-A7C-brighter_ISO100_sha1=18eaa9931d9a0f6f0511552ef6bf2fd040d82878.arw')


In [ ]:
imdng = rhdng.as_rgb()

In [ ]:
width, height = rh.raw.shape

left = (width - crop_size) // 2
top = (height - crop_size) // 2

if left % 2 != 0:
    left -= 1
if top % 2 != 0:
    top -= 1

right = left + crop_size
bottom = top + crop_size

im = rh.as_rgb(dims=(left, right, top, bottom))

In [ ]:
(imdng-im).mean()